# Introducción

En este notebook, se implementa [YOLOv4](https://arxiv.org/pdf/2004.10934.pdf) para el entrenamiento de una dataset propia en PyTorch con el objetivo de detectar personas que usan máscaras en las calles.

Se tomarán los siguientes pasos para implementar YOLOv4 en la dataset ya mencionada:
* Anteriormente, en los pasos 1 y 2, se extrajo las imagenes y se dio el formato adecuado del dataset
* Subi el dataset a la plataforma Roboflow para el correcto formato de train/ test/ valid
* Preparar el entorno YOLOv4 prentrenado(con los pesos obtenidos en la dataset COCO).
* Entrenar un detector personalizado YOLOv4 (se personaliza la última capa de este modelo para que se ajuste a nuestro objetivo).
* Volver a cargar los pesos del detector personalizado YOLOv4 para las predicciones(en este caso se usará la porción de imágenes de prueba y un video para medir la eficiencia del nuevo modelo con sus nuevos pesos).

Para todo este proposito, se uso google drive para compartir los archivos del dataset

Se desea predecir lo siguiente:

### Clase 1 - gente sin máscaras

### Clase 0 - gente con máscaras

#### ![people with mask](https://ichef.bbci.co.uk/news/320/cpsprodpb/3EFD/production/_111552161_mediaitem111552159.jpg)


#### ![people without mask](https://i.ytimg.com/vi/NyLF8nHIquM/maxresdefault.jpg)



# Configuramos YOLOV4

Se usa el modelo [YOLOv4Roboflow](https://github.com/roboflow-ai/pytorch-YOLOv4.git) como base de la detección de rostros con y sin máscara.

.Primero, se clona la arquitectura YOLOv4, esta es una pequeña versión pero bastante útil.

.Segundo, se cargan los pesos prentrenados en el dataset COCO, además solo se ajusta la última capa según los objetivos del presente trabajo. Bajo esta premisa, se especifica el número de clases y bounding box.


In [1]:
#clone a pretrained model 
!git clone https://github.com/roboflow-ai/pytorch-YOLOv4.git

Cloning into 'pytorch-YOLOv4'...
remote: Enumerating objects: 392, done.
remote: Total 392 (delta 0), reused 0 (delta 0), pack-reused 392
Receiving objects: 100% (392/392), 914.63 KiB | 13.07 MiB/s, done.
Resolving deltas: 100% (223/223), done.


In [2]:
#install all packages
%cd /content/pytorch-YOLOv4
!pip install -r requirements.txt

/content/pytorch-YOLOv4
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 12.6MB 232kB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 2.1MB 28.2MB/s 
     |████████████████████████████████| 317kB 41.5MB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.3 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 2.2.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
   

In [3]:
# download yolov4 weights that have already been converted to Python
!gdown https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA

Downloading...
From: https://drive.google.com/uc?id=1fcbR0bWzYfIEdLJPzOsn4R5mlvR6IQyA
To: /content/pytorch-YOLOv4/yolov4.conv.137.pth
170MB [00:01, 166MB/s]


# Dataset

Se usó el siguiente link https://app.roboflow.ai/ds/ScPwmjb6R3?key=f9e4IUDLIB como guía para manipular la data. Se usó roboflow para dar el formato correcto ya que el modelo necesita tener la data de manera uniforme de la siguiente manera: "nombre_del_archivo.jpg x, y, x+offset, y+offset, clase" y se repite de esta manera. 

| Nombre | BBox   |
|------|------|
|   imagen_1.jpg  | x1,y1,x2,y2,clase x1,y1,x2,y2,clase|
|   imagen_2.jpg  | x1,y1,x2,y2,clase|
|   imagen_3.jpg  | x1,y1,x2,y2,clase x1,y1,x2,y2,clase|
|   imagen_4.jpg  | x1,y1,x2,y2,clase x1,y1,x2,y2,clase x1,y1,x2,y2,clase|
|   ...  | ...|

Primero, para etiquetar a las imágenes de personas sin máscaras, se usa un modelo convolucional prentrenado (Multi Task Cascade Neural Network - step1). 

Segundo, se usan scripts diferentes para extraer de internet imágenes de personas con máscaras. Estas imágenes son de diferentes tamaños y RGB, por ello se reajustará el tamaño de estas. Con estas imágenes, se etiquetan a las personas que están usando máscaras usando la herramienta LabelImg. Con esta herramienta se realizó los bounding boxes de manera manual. Esto genera archivos XML con las anotaciones de los puntos superior izquierdo e inferior derecho. Seguidamente, se usa un script para colocar la data en el formato anteriormente mencionado. 

Finalmente, para facilitar el preprocesamiento, se usa roboflow para separar la data en tres conjuntos: train (70%), valid(20%) y test(20%).

Uno de los problemas encontrados con este procedimiento es que la cantidad de imágenes de personas con máscaras y la cantidad imágenes de personas sin máscara son disparejas , esto puede afectar en el entrenamiento del modelo YOLOv4.


Class Balance
* '#' hace referencia a la cantidad de bounding box encontradas en todas las imagenes, no a la cantidad de imagenes


| Class | # |
|------|------|
|   1 - sin macarilla | 3,234|
|   0 - con mascarilla  | 1,342|




In [4]:
!curl -L "https://app.roboflow.ai/ds/ScPwmjb6R3?key=f9e4IUDLIB" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   896  100   896    0     0   1331      0 --:--:-- --:--:-- --:--:--  1331
100 17.2M  100 17.2M    0     0  13.0M      0  0:00:01  0:00:01 --:--:-- 41.0M
Archive:  roboflow.zip
 extracting: test/143_jpg.rf.06b7389c7ba86de1d013d5c1ff4487cb.jpg  
 extracting: test/221_JPEG.rf.066bec23339041981e44ce7cf7ab0374.jpg  
 extracting: test/331_maxresdefault_jpg.rf.067a21bb4dd030cd30ebc7a3fea219b7.jpg  
 extracting: test/454_st-petersburg-russia-may-young-people-walking-along-nevsky-prospect-most-famous-street-34531504_jpg.rf.160e5bb6dbfc8a3b1f16ed725c053e32.jpg  
 extracting: test/51_JPEG.rf.20d16c77b4e974c973a3098841d290a1.jpg  
 extracting: test/148_jpg.rf.1cb2d362baf5056a5c547e1c178eadfd.jpg  
 extracting: test/295_JPEG.rf.10d14fa0faa7b26257305a8aeae6318e.jpg  
 extracting: test/290_maxresdefault_jpg.rf.353d5d7661b7a6f5ef30d57e936836

In [5]:
%cp train/_annotations.txt train/train.txt
%cp train/_annotations.txt train.txt
%cp valid/_annotations.txt data/val.txt
%cp valid/*.jpg train/

In [12]:
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_classes.txt')

In [13]:
print(num_classes)

2


# Entrenamiento del modelo 
Se usa una implementación de YOLOv4 en Pytorch. Se ajusta el proceso de entrenamiento a nuestra dataset, la cual tiene dos clases: 1(gente sin máscara) y 0(gente con máscara). 

Se presta atención a la pérdida total. Básicamente, esto es una combinación de 4 pérdidas: pérdida generada por los puntos x e y, pérdida en la altura y el ancho(MSE) y pérdida si los bounding box corresponden a una clase.

Primero, se cargan los pesos del modelo prentrenado con el dataset COCO y solo se actualiza la última capa que detecta los bounding boxes y su clase.

Búsqueda aleatoria: se usan diferentes tazas de aprendizaje para el entrenamiento del modelo.

El modelo que tiene la menor pérdida es el que se usará para detectar rostros con y sin máscaras en un video.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from tensorboardX import SummaryWriter
import logging
import os, sys
from tqdm import tqdm
from tool.utils import *
from dataset import Yolo_dataset
from cfg import Cfg
from models import Yolov4
import argparse
from easydict import EasyDict as edict
from torch.nn import functional as F
import time
from stat import S_ISREG, ST_CTIME, ST_MODE
from train import *
from models import *

import glob
import cv2

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Se habilitan cudas para acelerar el entrenamiento.

In [9]:
torch.cuda.is_available()

True

Para personalizar el modelo configuramos el archivo cfg. Entonces, definimos los hiperparámetros que se usarán en el modelo en este archivo durante el  entrenamiento: 

* Tamaño del batch (batch_size): 2
* Valores de la tasa de aprendizaje (lr): [0.0005, 0.001, 0.003, 0.005, 0.01] 
* Número de clases (n_clases): 2  
* Épocas (epochs): 50

Definimos una función personalizada 'train_yolo_custom', acá se cargarán las configuraciones hechas al archivo cfg. También se hace un resize de las imágenes con 'img_scale'.

Antes de comenzar con el entrenamiento, se cargan los pesos obtenidos con la dataset COCO(transferencia de aprendizaje).

También se usa un optimizador Adam y un actualizador de la tasa de aprendizaje (scheduler). Para las pérdidas se usara la función 'Yolo_loss', ya implementada en los archivos en los que nos basamos, y el error se guardará en 'loss'. Se habilita la opción de checkpoints para ir guardando periódicamente los pesos del entrenamiento, esto se hará cada 10 épocas.

In [10]:
def train_yolo_custom(name_lr, model, device, config, epochs=5, batch_size=1, save_cp=True, log_step=20, img_scale=0.5):
    train_dataset = Yolo_dataset(config.train_label, config)
    val_dataset = Yolo_dataset(config.val_label, config)

    n_train = len(train_dataset)
    n_val = len(val_dataset)

    train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                              num_workers=8, pin_memory=True, drop_last=True, collate_fn=collate)

    val_loader = DataLoader(val_dataset, batch_size=config.batch // config.subdivisions, shuffle=True, num_workers=8,
                            pin_memory=True, drop_last=True)

    writer = SummaryWriter(log_dir=config.TRAIN_TENSORBOARD_DIR,
                           filename_suffix=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}',
                           comment=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}')

    max_itr = config.TRAIN_EPOCHS * n_train

    global_step = 0
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {config.batch}
        Subdivisions:    {config.subdivisions}
        Learning rate:   {config.learning_rate}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images size:     {config.width}
        Optimizer:       {config.TRAIN_OPTIMIZER}
        Dataset classes: {config.classes}
        Train label path:{config.train_label}
        Pretrained:
    ''')

    # learning rate setup
    def burnin_schedule(i):
        if i < config.burn_in:
            factor = pow(i / config.burn_in, 4)
        elif i < config.steps[0]:
            factor = 1.0
        elif i < config.steps[1]:
            factor = 0.1
        else:
            factor = 0.01
        return factor

    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate / config.batch, betas=(0.9, 0.999), eps=1e-08)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, burnin_schedule)

    criterion = Yolo_loss(device=device, batch=config.batch // config.subdivisions,n_classes=config.classes)

    model.train()
    for epoch in range(epochs):
        #model.train()
        epoch_loss = 0
        epoch_step = 0

        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs} when lr is {name_lr}', unit='img', ncols=50) as pbar:
            for i, batch in enumerate(train_loader):
                global_step += 1
                epoch_step += 1
                images = batch[0]
                bboxes = batch[1]

                images = images.to(device=device, dtype=torch.float32)
                bboxes = bboxes.to(device=device)

                bboxes_pred = model(images)
                loss, _, _, _, _, _ = criterion(bboxes_pred, bboxes)

                loss.backward()

                epoch_loss += loss.item()

                if global_step  % config.subdivisions == 0:
                    optimizer.step()
                    scheduler.step()
                    model.zero_grad()

                if global_step % (log_step * config.subdivisions) == 0:
                    writer.add_scalar('train/Loss', loss.item(), global_step)
                    writer.add_scalar('lr', scheduler.get_lr()[0] * config.batch, global_step)
                    '''
                    pbar.set_postfix({'loss (batch)': loss.item(), 'loss_xy': loss_xy.item(),
                                        'lr': scheduler.get_lr()[0] * config.batch
                                        })
                    '''
                    logging.info('Train step_{}: loss : {},lr : {}'
                                  .format(global_step, loss.item(),
                                          scheduler.get_lr()[0] * config.batch))

                pbar.update(images.shape[0])

            if save_cp and ((epoch+1)%10)==0:
                try:
                    os.mkdir(config.checkpoints)
                    logging.info('Created checkpoint directory')
                except OSError:
                    pass
                torch.save(model.state_dict(), os.path.join(config.checkpoints, f'Yolov4_lr_{name_lr}_epoch{epoch + 1}.pth'))
                logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()


In [11]:
def get_custom_args(batch_size, lr, n_clases, epochs, **kwargs):
    cfg = edict(kwargs)

    cfg.batch = batch_size
    cfg.subdivisions = 1
    cfg.learning_rate = lr
    cfg.load = None 
    cfg.gpu = '0'
    cfg.dataset_dir = './train'
    cfg.pretrained = './yolov4.conv.137.pth'
    cfg.classes = n_clases 
    cfg.train_label = 'train.txt'
    cfg.TRAIN_EPOCHS = epochs 


    return edict(cfg)

In [12]:
batch_size = 2
epochs = 50
n_clases = num_classes

list_lr = [0.0005, 0.001, 0.003, 0.005, 0.01]
#list_lr = [0.001, 0.01]

Se llama a la función 'get_custom_args' que tendrá como parámetros de entrada las constantes definidas. En general, el parametro mas importante son los pesos cargados por un modelo prentrenado en COCODATASET.

También se activa los cuda cores para acelerar el entrenamiento.

Se define el modelo a usar 'model' con las configuraciones hechas y se procede a llamar la función personalizada 'train_yolo_custom' para comenzar con el entrenamiento del modelo.

Debido a que el entrenamiento puede tomar horas definimos un 'try-except' para poder interrumpir el proceso y guardar los pesos que se han conseguido hasta ese momento.

In [ ]:
for lr in list_lr:
    logging = init_logger(log_dir='log')
    
    # Cfg, argumentos dados por el modelo en el que nos estamos basando
    custom_config = get_custom_args(batch_size, lr, n_clases, epochs, **Cfg)
    
    os.environ["CUDA_VISIBLE_DEVICES"] = custom_config.gpu
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Using device {device}')

    model = Yolov4(custom_config.pretrained,n_classes=custom_config.classes)

    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    model.to(device=device)

    try:
        train_yolo_custom(name_lr = lr,
              model=model,
              config=custom_config,
              epochs=custom_config.TRAIN_EPOCHS,
              device=device, )
    except KeyboardInterrupt:
        torch.save(model.state_dict(), 'INTERRUPTED.pth')
        logging.info('Saved interrupt')
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)

  

log file path:log/log_2020-07-16_13-12-14.txt
2020-07-16 13:12:14,840 <ipython-input-13-349ba4c45348>[line:9] INFO: Using device cuda
2020-07-16 13:12:18,051 <ipython-input-10-8b817d6efbfb>[line:35] INFO: Starting training:
        Epochs:          50
        Batch size:      2
        Subdivisions:    1
        Learning rate:   0.001
        Training size:   601
        Validation size: 172
        Checkpoints:     True
        Device:          cuda
        Images size:     608
        Optimizer:       adam
        Dataset classes: 2
        Train label path:train.txt
        Pretrained:
    


Epoch 1/50 when lr is 0.001:   0%| | 0/601 [00:00</usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
Epoch 1/50 when lr is 0.001:   6%| | 38/601 [00:18

2020-07-16 13:12:37,739 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_20: loss : 32970.12890625,lr : 1.6000000000000002e-10


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Epoch 1/50 when lr is 0.001:  13%|▏| 78/601 [00:36

2020-07-16 13:12:55,134 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_40: loss : 32903.88671875,lr : 2.5600000000000003e-09


...

2020-07-16 16:53:44,518 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_60: loss : 28758.029296875,lr : 1.2959999999999997e-07
2020-07-16 16:53:44,518 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_60: loss : 28758.029296875,lr : 1.2959999999999997e-07


Epoch 1/50 when lr is 0.01:  26%|▎| 158/601 [01:11

2020-07-16 16:54:01,841 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_80: loss : 28703.455078125,lr : 4.096e-07
2020-07-16 16:54:01,841 <ipython-input-10-8b817d6efbfb>[line:92] INFO: Train step_80: loss : 28703.455078125,lr : 4.096e-07


Epoch 1/50 when lr is 0.01:  28%|▎| 168/601 [01:15Exception in thread Thread-108:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 294, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=proces

2020-07-16 16:54:06,593 <ipython-input-13-349ba4c45348>[line:25] INFO: Saved interrupt


## Cambiar directorio y guardar pesos 

In [2]:
!pwd

/content


In [3]:
%cd pytorch-YOLOv4/

/content/pytorch-YOLOv4


In [4]:
#looking at the weights that our model has saved during training
!ls checkpoints

Yolov4_lr_0.001_epoch10.pth  Yolov4_lr_0.001_epoch40.pth
Yolov4_lr_0.001_epoch20.pth  Yolov4_lr_0.001_epoch50.pth
Yolov4_lr_0.001_epoch30.pth


In [5]:
!zip -r checkpoints_lr_0.001.zip checkpoints

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/Yolov4_lr_0.001_epoch10.pth (deflated 7%)
  adding: checkpoints/Yolov4_lr_0.001_epoch30.pth (deflated 7%)
  adding: checkpoints/Yolov4_lr_0.001_epoch40.pth (deflated 7%)
  adding: checkpoints/Yolov4_lr_0.001_epoch20.pth (deflated 7%)
  adding: checkpoints/Yolov4_lr_0.001_epoch50.pth (deflated 7%)


In [6]:
!cp /content/pytorch-YOLOv4/checkpoints_lr_0.001.zip '/content/drive/My Drive/DEEP_LEARNING_MAESTRIA_PUCP/proyecto'

In [7]:
!pwd

/content/pytorch-YOLOv4


# Testing model

Probamos el modelo obtenido con el dataset de prueba. Esta dataset es organizada por roboflow.

Primero, se define una función 'detect_bounding_boxes' que recibirá de entrada el directorio donde se encuentran almacenadas las imágenes de prueba, el directorio donde se almacenarán las predicciones, el número de imágenes, un arreglo donde se encuentra el número de clases junto con los pesos del modelo y un bloc de notas con las clases de la dataset de prueba.

Antes de predecir el resultado, se hace un resize a las imágenes de entrada y se convierte todas las imágenes a un formato RGB. 

Finalmente, despues de obtener las inferencias, se abren estas imágenes para tener un conocimiento breve sobre los resultados.

Para reconocer la clase de los bounding boxes se usará la siguiente distinción:

* Con mascarilla (clase 0): color violeta.
* Sin mascarilla (clase 1): color verde.

Los bounding boxes obtenidos junto con sus respectivas clasificaciones se guardarán en un nuevo directorio '/predictions'.

In [8]:
def detect_bounding_boxes(path,args,number_images, path_target):
  
  files = glob.glob(os.path.expanduser(path + '*'))
  # Sort by modification time (mtime) ascending and descending
  path_sorted = sorted(files, key=lambda t: os.stat(t).st_mtime)

  for filename in path_sorted[:number_images]:

    filename = filename.split('/')[1]

    if not filename.endswith('.jpg'):
      continue
    
    namesfile = None


    if len(args) == 2:
        n_classes = int(args[0])
        weightfile = args[1]
        imgfile = path + filename
    elif len(args) == 3:
        n_classes = int(args[0])
        weightfile = args[1]
        imgfile = path + filename
        class_names_file = args[2]

    model = Yolov4(n_classes=n_classes)

    pretrained_dict = torch.load(weightfile, map_location=torch.device('cuda'))
    model.load_state_dict(pretrained_dict)

    use_cuda = 1
    if use_cuda:
        model.cuda()

    img = Image.open(imgfile).convert('RGB')
    sized = img.resize((608, 608))

    boxes = do_detect(model, sized, 0.5, n_classes,0.4, use_cuda)

    class_names = load_class_names(class_names_file)

    dir_path = os.path.dirname(path_target)
    if not os.path.exists(dir_path):
      os.makedirs(dir_path)

    plot_boxes(img, boxes, path_target + filename[:-4] + '_predict.jpg', class_names)

Se definen los argumentos a usar en el conjunto de prueba. En este caso, comparando los resultados a partir del loss, se opto por usar los pesos con una configuracion del lr de 0.001 en la epoca 50 (a partir de los checkpoints del modelo). Este parametro se pude cambiar

In [9]:
args = {}
args[0] = 2 #n classes
args[1] = 'checkpoints/Yolov4_lr_0.001_epoch50.pth' #weight file
args[2] = 'test/_classes.txt' #test classes file

Definimos la dirección donde se encuentran nuestras imágenes de prueba 'path_test' y también la dirección donde se almacenarán las predicciones 'path_target_test'.
 

In [14]:
path_test = 'test/'
number_images = -1
path_target_test = 'predictions/'
detect_bounding_boxes(path_test,args, number_images, path_target_test)

0: 1.000000
save plot results to predictions/220_JPEG.rf.ee6affb27b0c6f8f4981af8dc74287f6_predict.jpg
save plot results to predictions/144_jpg.rf.ac885759a2df17771120628d3ab932a8_predict.jpg
save plot results to predictions/178_jpg.rf.aee9a1e5e4a5a1e7cc472ce3342f30b9_predict.jpg
0: 1.000000
0: 0.999921
0: 0.999326
0: 0.999998
save plot results to predictions/204_JPEG.rf.fde870ac55e8f2aab613f0705a95d76e_predict.jpg
0: 1.000000
0: 0.999998
0: 0.999999
save plot results to predictions/514_JPEG.rf.7ad28235ab02c0f48e4c3735855e335f_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000


/content/pytorch-YOLOv4/tool/utils.py:16: RuntimeWarning: overflow encountered in exp
  return 1.0 / (np.exp(-x) + 1.)


0: 0.999936
save plot results to predictions/143_jpg.rf.06b7389c7ba86de1d013d5c1ff4487cb_predict.jpg
0: 0.999995
0: 0.999992
save plot results to predictions/37_PNG.rf.924187d33d8186d0ddb065f05464855f_predict.jpg
1: 1.000000
1: 0.999997
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 0.984092
1: 0.999990
1: 0.999998
1: 1.000000
0: 0.999979
0: 0.999967
1: 0.999971
1: 0.999972
1: 1.000000
0: 0.999517
0: 0.513803
save plot results to predictions/66_JPEG.rf.57a1f8af492ee60deb195716801caaf8_predict.jpg
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 1.000000
0: 0.999997
0: 1.000000
1: 1.000000
1: 1.000000
0: 1.000000
1: 1.000000
1: 0.999442
1: 1.000000
0: 1.000000
1: 0.999317
save plot results to predictions/349_JPEG.rf.7b909ba8b9eafeb3c40a6a30a728ef0e_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/536_street-market-kota-bharu-jun-local-people-shopping-crowded-markets-famous-rantau-panjang-ju

In [16]:
#visualize inference
#show images 
# se infieren los resultados en 30 imagenes

path_detected_images = 'predictions/'
number_images_to_show = 30

for filename in os.listdir(path_detected_images)[:number_images_to_show]:
  img = cv2.imread(path_detected_images + filename)
  img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
  print(img.shape)
  plt.imshow(img)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [17]:
!pwd

/content/pytorch-YOLOv4


In [18]:
#liberar memoria de disco
!rm -r {path_detected_images}

# Detectar personas con máscara en un video

Para la detección de personas con máscaras en un video se usará el video [Americans face a coronavirus mask dilemma](https://www.youtube.com/watch?v=g-dyHadto2Y).
La duracion del video es de 2 minutos aproximadamente

Este video es un buenc conjunto de imagenes a evaluar, ya que entre frame y frame aparecen personas con macarilla y sin mascarilla. Ademas, se puede apreciar a las personas en primer plano y en un plano mas panoramico

Se define una función 'getFrame' para obtener frames del video mencionado cada cierto tiempo configurable

In [19]:
#download a video which is in google drive the url is https://www.youtube.com/watch?v=g-dyHadto2Y

!cp '/content/drive/My Drive/DEEP_LEARNING_MAESTRIA_PUCP/proyecto/americans-face-a-coronavirus-mask-dilemma.mp4' /content/pytorch-YOLOv4


In [40]:
#remove directories just in case of doubt
!rm -r object_detection_video
!rm -r predictions

In [32]:
def getFrame(sec,source_video):
    source_video.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = source_video.read()

    dir_path = os.path.dirname('object_detection_video/')
    if not os.path.exists(dir_path):
      os.makedirs(dir_path)

    if hasFrames:
        cv2.imwrite('object_detection_video/' + 'frame_' + str(sec) + '_sec.jpg', image)     # save frame as JPG file
    
    return hasFrames

Se definen los frames por segundo que se desean. En este caso, para fines de prueba del modelo, se ha tomado 5 fps lo cual equivale a un ratio de 0.2. Es decir se generaran 5 imagenes cada segundo a partir del video

In [42]:
name_file_video = 'americans-face-a-coronavirus-mask-dilemma.mp4' 

source_video = cv2.VideoCapture(name_file_video)

sec = 0
frames_per_second_videoin = 5
frameRate = 1/ frames_per_second_videoin
success = getFrame(sec, source_video)
while success:
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec, source_video)

In [43]:
args = {}
args[0] = 2 #n classes
args[1] = 'checkpoints/Yolov4_lr_0.001_epoch50.pth' #weight file
args[2] = 'test/_classes.txt' #test classes file

Finalmente, se usara el modelo usando las imagenes adquiridass del video para detectar a las personas.

Recordar que los bounding boxes de las personas que usan mascarillas(clase 0) son de color violeta mientras que los bounding boxes de las personas sin mascarilla (clase 1) son de color verde.

In [44]:
path_video = 'object_detection_video/'
number_images = -1
path_detected_images = 'predictions/'
detect_bounding_boxes(path_video,args, number_images, path_detected_images)

/content/pytorch-YOLOv4/tool/utils.py:16: RuntimeWarning: overflow encountered in exp
  return 1.0 / (np.exp(-x) + 1.)


save plot results to predictions/frame_0_sec_predict.jpg
1: 1.000000
1: 0.999997
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/frame_0.2_sec_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/frame_0.4_sec_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/frame_0.6_sec_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/frame_0.8_sec_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
save plot results to predictions/frame_1.0_sec_predict.jpg
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000
1: 1.000000


De los datos obtenidos, se genera un video con los boundingboxes incluidos. Este video será almacenadoen la carpeta de 'predictions/'. Este video tiene la misma duracion que el video fuente pero a 5 fps, que fue la configuracion que se tomo en los pasos anteriores. Finalmente se guarda el resultado del video en el drive compartido por el equipo

In [45]:
#saving in video named  "'detected_masks_video.avi'"
img_array = []

frames_per_second = frames_per_second_videoin

path_predictions = 'predictions/'

#sortfiles
files = glob.glob(os.path.expanduser(path_predictions + '*'))

path_sorted = sorted(files, key=lambda t: os.stat(t).st_mtime)

for filename in path_sorted[:number_images]:
    filename = filename.split('/')[1]
    img = cv2.imread(path_predictions + filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('detected_masks_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), frames_per_second, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [46]:
!cp /content/pytorch-YOLOv4/detected_masks_video.avi '/content/drive/My Drive/DEEP_LEARNING_MAESTRIA_PUCP/proyecto'


# Fuentes
* Paper YOLOv4 https://arxiv.org/pdf/2004.10934.pdf
* Github del cual adaptamos el codigo existente: https://github.com/roboflow-ai/pytorch-YOLOv4.git
* Herramienta para poder dividir el dataset y darle el formato adecuado https://roboflow.ai/